In [1]:
!huggingface-cli download TheBloke/zephyr-7B-beta-GGUF zephyr-7b-beta.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:141: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
zephyr-7b-beta.Q4_K_M.gguf: 100% 4.37G/4.37G [00:27<00:00, 159MB/s]
Download complete. Moving file to zephyr-7b-beta.Q4_K_M.gguf
zephyr-7b-beta.Q4_K_M.gguf


In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on"

In [3]:
!pip install llama-cpp-python -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 17.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00


In [5]:
from llama_cpp import Llama


llm = Llama(
      model_path="/content/zephyr-7b-beta.Q4_K_M.gguf",
      n_gpu_layers=-1, # Uncomment to use GPU acceleration
      seed=1337, # Uncomment to set a specific seed
      n_ctx=2048, # Uncomment to increase the context window
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

In [6]:
llm("What is large Language Model?", max_tokens=128)

llama_perf_context_print:        load time =    2248.72 ms
llama_perf_context_print: prompt eval time =    2248.42 ms /     7 tokens (  321.20 ms per token,     3.11 tokens per second)
llama_perf_context_print:        eval time =   73681.60 ms /   127 runs   (  580.17 ms per token,     1.72 tokens per second)
llama_perf_context_print:       total time =   75998.36 ms /   134 tokens
llama_perf_context_print:    graphs reused =        122


{'id': 'cmpl-964d1dc9-8234-4f63-a793-5a50e5e47108',
 'object': 'text_completion',
 'created': 1755523992,
 'model': '/content/zephyr-7b-beta.Q4_K_M.gguf',
 'choices': [{'text': '\n\nIn a general sense, a language model is a statistical model that predicts the probability of a sequence of words. In a more specific sense, a language model is the core of a text generation system, such as Google’s DeepMind’s GPT (Generative Pre-trained Transformer) and OpenAI’s GPT-2. In this article, we will mainly focus on GPT.\n\nGPT is a powerful language model with over 1.5 billion parameters, trained on a vast corpus of text data. The model is capable of generating human-like text and can be fine',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 7, 'completion_tokens': 128, 'total_tokens': 135}}

In [1]:
!pip install llama-index-llms-llama-cpp gradio

In [2]:
from typing import Sequence, Optional

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core.llms import ChatMessage


# definisi completion_to_prompt manual
def completion_to_prompt(completion: str) -> str:
    return f"<|assistant|>\n{completion}</s>\n"


In [3]:
def zephyr_messages_to_prompt(
    messages: Sequence[ChatMessage],
    system_prompt: Optional[str] = None
) -> str:
    prompt = ""
    for message in messages:
        prompt += f"<|{message.role}|>\n"
        prompt += f"{message.content}</s>\n"

    return prompt + "<|assistant|>\n"


In [4]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=None,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path="/content/zephyr-7b-beta.Q4_K_M.gguf",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=zephyr_messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

In [32]:
messages = [
    ChatMessage(
        role="system",
        content="You are a chatbot who always responds concise and pleasant"
    ),
    ChatMessage(
        role="user",
        content="There's a llama on my lawn, how can I get rid of him?"
    )
]

# zephyr_messages_to_prompt(messages)

response = llm.stream_chat(messages)
for r in response:
  print(r.delta, end="", flush=True)

I'm sorry to hear that. Here are a few suggestions to help you get rid of the llama:

1. Contact your local animal control agency or wildlife rehabilitation center for assistance. They may be able to provide guidance on how to safely and humanely remove the llama from your property.

2. Try to lure the llama away with food or water. This may help convince it to leave on its own.

3. Use a loud noise or bright light to startle the llama and encourage it to move along.

4. If the llama is causing damage to your property, you may need to consider hiring a professional llama wrangler or animal removal service.

5. In the future, consider installing a fence or other barrier to prevent llamas and other animals from entering your property.

Remember to always prioritize the safety and well-being of the animal, as well as your own safety.

llama_perf_context_print:        load time =   15820.24 ms
llama_perf_context_print: prompt eval time =   15819.86 ms /    63 tokens (  251.11 ms per token,     3.98 tokens per second)
llama_perf_context_print:        eval time =  135317.66 ms /   202 runs   (  669.89 ms per token,     1.49 tokens per second)
llama_perf_context_print:       total time =  151567.82 ms /   265 tokens
llama_perf_context_print:    graphs reused =        194


In [5]:
import gradio as gr


def generate_response(message, history):
    chat_messages = [
        ChatMessage(
            role="system",
            content="You are a chatbot who always responds concise and pleasant"
        ),
    ]
    for human, ai in history:
        chat_messages.append(ChatMessage(
            role="user",
            content=human
        ))
        chat_messages.append(ChatMessage(
            role="assistant",
            content=ai
        ))

    chat_messages.append(ChatMessage(
        role="user",
        content=message
    ))

    response = llm.stream_chat(chat_messages)
    text = ""
    for r in response:
      text += r.delta
      yield text

gr.ChatInterface(generate_response).launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fb75c1bf0ffc9406dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
